In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Download the data

In [18]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [19]:
concrete_data.shape

(1030, 9)

In [20]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [21]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split data into predictors and target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [22]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [23]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [24]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [25]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [26]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

# Build a Neural Network


In [27]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)

In [29]:
# build the model
model = regression_model()

# Train and Test the Network


In [30]:
# build the model
model.fit( X_test, y_test,epochs=100, verbose=2)

Epoch 1/100
10/10 - 0s - loss: 44288.8203
Epoch 2/100
10/10 - 0s - loss: 23295.8203
Epoch 3/100
10/10 - 0s - loss: 11383.9141
Epoch 4/100
10/10 - 0s - loss: 4317.7329
Epoch 5/100
10/10 - 0s - loss: 1458.5317
Epoch 6/100
10/10 - 0s - loss: 691.3572
Epoch 7/100
10/10 - 0s - loss: 451.1318
Epoch 8/100
10/10 - 0s - loss: 360.5255
Epoch 9/100
10/10 - 0s - loss: 313.4105
Epoch 10/100
10/10 - 0s - loss: 293.9106
Epoch 11/100
10/10 - 0s - loss: 281.5162
Epoch 12/100
10/10 - 0s - loss: 278.7816
Epoch 13/100
10/10 - 0s - loss: 276.1413
Epoch 14/100
10/10 - 0s - loss: 273.4377
Epoch 15/100
10/10 - 0s - loss: 271.5561
Epoch 16/100
10/10 - 0s - loss: 269.2380
Epoch 17/100
10/10 - 0s - loss: 267.1916
Epoch 18/100
10/10 - 0s - loss: 264.9340
Epoch 19/100
10/10 - 0s - loss: 263.1215
Epoch 20/100
10/10 - 0s - loss: 262.0113
Epoch 21/100
10/10 - 0s - loss: 261.3308
Epoch 22/100
10/10 - 0s - loss: 259.6798
Epoch 23/100
10/10 - 0s - loss: 258.4943
Epoch 24/100
10/10 - 0s - loss: 257.9185
Epoch 25/100
10/1

Model evaluation

In [31]:
loss = model.evaluate(X_test, y_test)
y_hat = model.predict(X_test)
loss

10/10 [==============================] - 0s 1ms/step - loss: 195.4676


195.4675750732422

Create a list of mean squared errors and  the standard deviation of the mean squared errors


In [32]:
total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0,total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=4)
    model.fit(X_train, y_train, epochs=100, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_hat = model.predict(X_test)
    errors = mean_squared_error(y_test, y_hat)
    mean_squared_errors.append(errors)
    
mean = np.mean(np.array(mean_squared_errors))
STD = np.std(mean_squared_errors)
print("Mean: ",str(mean), "Standard Deviation:",str(STD))


MSE 1: 122.6963119506836
MSE 2: 74.27244567871094
MSE 3: 51.95005798339844
MSE 4: 46.339576721191406
MSE 5: 50.00593185424805
MSE 6: 49.893348693847656
MSE 7: 43.67530822753906
MSE 8: 48.20466613769531
MSE 9: 49.37314224243164
MSE 10: 43.20817565917969
MSE 11: 43.151371002197266
MSE 12: 43.52834701538086
MSE 13: 43.108848571777344
MSE 14: 44.28691482543945
MSE 15: 45.20608139038086
MSE 16: 43.635581970214844
MSE 17: 43.65042495727539
MSE 18: 45.063331604003906
MSE 19: 44.91679000854492
MSE 20: 46.18994140625
MSE 21: 45.6699333190918
MSE 22: 46.57896041870117
MSE 23: 43.611412048339844
MSE 24: 44.372344970703125
MSE 25: 42.06134796142578
MSE 26: 41.1082763671875
MSE 27: 39.66709518432617
MSE 28: 39.060489654541016
MSE 29: 38.3679084777832
MSE 30: 38.258121490478516
MSE 31: 41.17998123168945
MSE 32: 39.29533004760742
MSE 33: 41.893104553222656
MSE 34: 39.472816467285156
MSE 35: 39.24471664428711
MSE 36: 40.5914192199707
MSE 37: 40.14532470703125
MSE 38: 40.233951568603516
MSE 39: 43.0228